In [1]:
# This notebook connects to the LaundroLink database, analyzes customer behavior,
# groups customers into segments using K-Means clustering, and saves the
# results back to the database for the web dashboards to use.

# --- 1. Import Libraries ---
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import mysql.connector
from dotenv import load_dotenv
import os

print("--- Step 1: Libraries imported ---")

# --- 2. Database Connection ---
# Load database credentials securely from your .env file
# This assumes your .env file is in the parent 'Backend' folder
try:
    load_dotenv('../Backend/.env') 
    db = mysql.connector.connect(
        host=os.getenv("DB_HOST"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        database=os.getenv("DB_NAME")
    )
    print("--- Step 2: Database connection successful! ---")
except mysql.connector.Error as err:
    print(f"--- ❌ Step 2 FAILED: Error connecting to database: {err} ---")
    # Stop execution if the database connection fails
    exit()

# --- 3. Fetch Customer Behavior Data ---
# This query gathers all the raw data we need to understand customer behavior.
# It only includes customers who have at least one paid invoice.
query = """
SELECT
    c.CustID,
    COUNT(DISTINCT o.OrderID) AS order_frequency,
    SUM(i.PayAmount) AS total_spent,
    AVG(i.PayAmount) AS average_spend,
    DATEDIFF(NOW(), MAX(o.OrderCreatedAt)) as days_since_last_order
FROM Customer c
JOIN Orders o ON c.CustID = o.CustID
JOIN Invoice i ON o.OrderID = i.OrderID
WHERE i.InvoiceID IN (SELECT InvoiceID FROM Invoice_Status WHERE InvoiceStatus = 'Paid')
GROUP BY c.CustID
HAVING order_frequency > 0;
"""

df = pd.read_sql(query, db)
print("\n--- Step 3: Fetched Raw Customer Data ---")
print(df.head())


# --- 4. Data Preparation for Clustering ---
# We need to prepare the data so the clustering algorithm can understand it.
print("\n--- Step 4: Preparing data for clustering ---")

# Select only the numerical features for clustering
features = df[['order_frequency', 'total_spent', 'average_spend', 'days_since_last_order']]

# Scale the data. This is crucial for K-Means to work correctly.
# It ensures that one feature (like total_spent) doesn't dominate the others.
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)
print("Data scaled successfully.")


# --- 5. Perform K-Means Clustering ---
# We'll group the customers into 4 segments.
print("\n--- Step 5: Performing K-Means clustering ---")
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df['cluster'] = kmeans.fit_predict(scaled_features)
print("Clustering complete.")


# --- 6. Analyze and Name the Segments ---
# Let's analyze the characteristics of each cluster to give them meaningful names.
cluster_analysis = df.groupby('cluster')[['order_frequency', 'total_spent', 'average_spend', 'days_since_last_order']].mean().sort_values('total_spent', ascending=False)
print("\n--- Step 6: Cluster Analysis (Averages) ---")
print(cluster_analysis)

# Based on the analysis, we can define our segment names.
# This logic dynamically assigns names based on spending and recency.
cluster_map = {}
sorted_clusters = cluster_analysis.reset_index()

cluster_map[sorted_clusters.loc[0, 'cluster']] = 'High-Value Spenders'
cluster_map[sorted_clusters.loc[1, 'cluster']] = 'Loyal Regulars'

# Find the cluster with the highest days_since_last_order
at_risk_cluster = sorted_clusters.sort_values('days_since_last_order', ascending=False).iloc[0]['cluster']
if at_risk_cluster not in cluster_map:
    cluster_map[at_risk_cluster] = 'At-Risk Customers'

# The remaining cluster is 'New or Occasional'
for i in range(4):
    if i not in cluster_map:
        cluster_map[i] = 'New or Occasional'

df['segment_name'] = df['cluster'].map(cluster_map)
print("\nSegment names assigned:")
print(df[['CustID', 'segment_name']].head())


# --- 7. Save Results Back to the Database ---
# We'll create a new table to store our results.
print("\n--- Step 7: Saving results to the database ---")
cursor = db.cursor()

# Create the new table if it doesn't exist
cursor.execute("""
CREATE TABLE IF NOT EXISTS Customer_Segments (
    CustID VARCHAR(10) PRIMARY KEY,
    SegmentName VARCHAR(50),
    AnalyzedAt TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    FOREIGN KEY (CustID) REFERENCES Customer(CustID) ON DELETE CASCADE
);
""")
print("'Customer_Segments' table ready.")

# Insert or update the results for each customer
for index, row in df.iterrows():
    cursor.execute("REPLACE INTO Customer_Segments (CustID, SegmentName) VALUES (%s, %s)", (row['CustID'], row['segment_name']))

db.commit()
print(f"✅ Successfully updated segments for {len(df)} customers in the database.")

# Close the database connection
cursor.close()
db.close()


--- Step 1: Libraries imported ---
--- Step 2: Database connection successful! ---

--- Step 3: Fetched Raw Customer Data ---
    CustID  order_frequency  total_spent  average_spend  days_since_last_order
0  C25-005                3       1577.5     525.833333                      0
1  C25-006                2       1325.0     662.500000                      7
2  C25-007                3       1579.0     526.333333                      0
3  C25-008                1        312.0     312.000000                      3
4  C25-009                1        840.0     840.000000                      2

--- Step 4: Preparing data for clustering ---
Data scaled successfully.

--- Step 5: Performing K-Means clustering ---


C:\Users\Juriel\AppData\Local\Temp\ipykernel_22456\1693751035.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, db)


Clustering complete.

--- Step 6: Cluster Analysis (Averages) ---
         order_frequency  total_spent  average_spend  days_since_last_order
cluster                                                                    
0                    3.0      1578.25     526.083333                    0.0
2                    2.0      1325.00     662.500000                    7.0
1                    1.0       727.50     727.500000                    1.5
3                    1.0       312.00     312.000000                    3.0

Segment names assigned:
    CustID         segment_name
0  C25-005  High-Value Spenders
1  C25-006       Loyal Regulars
2  C25-007  High-Value Spenders
3  C25-008    New or Occasional
4  C25-009    New or Occasional

--- Step 7: Saving results to the database ---
'Customer_Segments' table ready.
✅ Successfully updated segments for 6 customers in the database.
